In [43]:
from tensorflow.keras.metrics import binary_accuracy, AUC, Precision, Recall
#Imports
import matplotlib.pyplot as plt
from math import ceil

import os
import pandas as pd
import numpy as np 
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers

from sklearn.metrics import accuracy_score


from sklearn.decomposition import PCA

In [ ]:
np.random.seed(1)
from tensorflow.keras.utils import set_random_seed
set_random_seed(1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/HackAthon

ls: cannot access '/content/drive/MyDrive/HackAthon': No such file or directory


In [ ]:
unzip_path = f'/content/drive/MyDrive/hotdog-nothotdog.zip'
!unzip -q "$unzip_path"

replace test/hotdog/1501.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [60]:
datagen = ImageDataGenerator(validation_split=0.2)

# get training data
train_ds = datagen.flow_from_directory(
    '/content/train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training',  # set as training data
    seed=42
)


# get validation data
val_ds = datagen.flow_from_directory(
    '/content/train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation',  # set as validation data
    seed=42,
    shuffle=False
)


# get test data
test_ds = datagen.flow_from_directory(
    '/content/test',
    target_size=(256, 256),
    class_mode='binary',
    seed=42
)

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 644 images belonging to 2 classes.


In [61]:
#Basic Model
model = Sequential()

model.add(layers.Input(shape=(256, 256, 3)))
model.add(layers.Rescaling(1./255))

# Convolutional and pooling layer
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))  # Adding dropout

# Add another
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))  # Adding dropout

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout

# Output
model.add(Dense(1, activation='sigmoid'))


In [78]:
from tensorflow.keras.metrics import binary_accuracy, AUC, Precision, Recall

In [79]:
standard_metrics = [
    'binary_accuracy', 
    'AUC', 
    'Precision', 
    'Recall'
]

In [80]:
model.compile(
    optimizer="adam",
    loss="bce",
    metrics=standard_metrics
)

In [81]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 127, 127, 32)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                  

In [82]:
fit= model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Epoch 1/10
75/75 [==============================] - 14s 169ms/step - loss: 0.3024 - binary_accuracy: 0.8675 - auc: 0.9427 - precision: 0.9038 - recall: 0.8225 - val_loss: 0.6773 - val_binary_accuracy: 0.7000 - val_auc: 0.7672 - val_precision: 0.7128 - val_recall: 0.6700
Epoch 2/10
75/75 [==============================] - 13s 168ms/step - loss: 0.2689 - binary_accuracy: 0.8779 - auc: 0.9547 - precision: 0.9290 - recall: 0.8183 - val_loss: 0.6856 - val_binary_accuracy: 0.6950 - val_auc: 0.7617 - val_precision: 0.7038 - val_recall: 0.6733
Epoch 3/10
75/75 [==============================] - 13s 168ms/step - loss: 0.2320 - binary_accuracy: 0.8942 - auc: 0.9663 - precision: 0.9171 - recall: 0.8667 - val_loss: 0.7557 - val_binary_accuracy: 0.7400 - val_auc: 0.8001 - val_precision: 0.7500 - val_recall: 0.7200
Epoch 4/10
75/75 [==============================] - 15s 196ms/step - loss: 0.2297 - binary_accuracy: 0.8967 - auc: 0.9673 - precision: 0.9281 - recall: 0.8600 - val_loss: 0.7737 - val_bin

In [70]:
standard_metrics

[<function keras.metrics.accuracy_metrics.binary_accuracy(y_true, y_pred, threshold=0.5)>,

In [83]:
export = []
for i in standard_metrics:
  export.append(fit.history[f'{i.lower()}'][-1])
  export.append(fit.history[f'val_{i.lower()}'][-1])
export

[0.9420833587646484,
 0.7149999737739563,
 0.9891034364700317,
 0.7653888463973999,
 0.9740840196609497,
 0.7362637519836426,
 0.9083333611488342,
 0.6700000166893005]

In [84]:
df = pd.DataFrame(data = np.array([export]), columns = ['train_accuracy','val_accuracy','train_AUC','val_AUC','train_Precision','val_Precision', 'train_Recall ','val_Recall'], index= ['convnet'])

In [91]:
df.to_csv('maia_hack.csv')

In [85]:
def model_preds(model):
  '''
  This function will return the y_true_classes and y_pred_clasess
  Which will allows us to build a confusion matrix
  '''
  
  y_true_classes = val_ds.labels

  y_pred_classes = model.predict(val_ds)

  y_pred_classes = np.argmax(y_pred_classes, axis=1)

  return (y_true_classes, y_pred_classes)

In [86]:
y_true_classes, y_pred_classes = model_preds(model)
#accuracy
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print("Accuracy:", accuracy)


19/19 [==============================] - 2s 116ms/step
Accuracy: 0.5
